# ChatBot with Memory

## Imports

In [1]:
# System
import os
os.environ['USER_AGENT'] = 'JimYin88'

# LLM Models
from langchain_openai import ChatOpenAI
from langchain_ollama import OllamaLLM
from langchain_ollama import ChatOllama

# Templates
from langchain_core.prompts import ChatPromptTemplate, MessagesPlaceholder
from langchain.schema import SystemMessage, HumanMessage, AIMessage

# Document Loaders
from langchain_community.document_loaders import Docx2txtLoader
from langchain_community.document_loaders import WebBaseLoader
# from langchain.document_loaders import BSHTMLLoader
# from langchain_community.document_loaders import UnstructuredRTFLoader

# Document Splitters
from langchain.text_splitter import TokenTextSplitter
# from langchain.text_splitter import RecursiveCharacterTextSplitter

# Embeddings
from langchain_google_genai import GoogleGenerativeAIEmbeddings
# from langchain_openai import OpenAIEmbeddings

# Vector Stores
# from langchain_chroma import Chroma
from langchain_community.vectorstores.faiss import FAISS

# LangChain Chains
from langchain.chains import create_retrieval_chain
from langchain.chains.combine_documents import create_stuff_documents_chain
from langchain.chains.history_aware_retriever import create_history_aware_retriever

# OutputParsers
from langchain.schema.output_parser import StrOutputParser

# Gradio
import gradio as gr

## Load Environment Variables

In [2]:
from dotenv import load_dotenv
load_dotenv()

True

In [3]:
# Retrieve Data
def get_docs(url):
    loader = WebBaseLoader(url)
    docs = loader.load()

    text_splitter = TokenTextSplitter(chunk_size=200,
                                      chunk_overlap=20)

    splitDocs = text_splitter.split_documents(docs)

    return splitDocs

In [4]:
def create_vector_store(docs):
    embedding_function = GoogleGenerativeAIEmbeddings(model="models/embedding-001")
    vector_store = FAISS.from_documents(documents=docs, 
                                        embedding=embedding_function)
    return vector_store

In [5]:
def create_chain(vector_store):
    
    chat_model = ChatOpenAI(model="gpt-4o-mini-2024-07-18",
                            max_completion_tokens=1028,
                            temperature=0.2)

    prompt = ChatPromptTemplate.from_messages([("system", "Answer the user's questions based on the context: {context}"),
                                               MessagesPlaceholder(variable_name="chat_history"),
                                               ("user", "{input}")])

    # chain = prompt | model
    document_chain = create_stuff_documents_chain(llm=chat_model, 
                                                  prompt=prompt, 
                                                  output_parser=StrOutputParser())
    
    retriever = vector_store.as_retriever(search_kwargs={'k': 2})

    retriever_prompt = ChatPromptTemplate.from_messages([
        MessagesPlaceholder(variable_name="chat_history"),
        ("user", "{input}"),
        ("user", "Given the above conversation, generate a search query to look up in order to get information relevant to the conversation")])
    
    history_aware_retriever = create_history_aware_retriever(llm=chat_model,
                                                             retriever=retriever,
                                                             prompt=retriever_prompt)

    retrieval_chain = create_retrieval_chain(
        # retriever, Replace with History Aware Retriever
        history_aware_retriever,
        document_chain)

    return retrieval_chain

In [6]:
def process_chat(chain, question, chat_history):
    
    response = chain.invoke({"input": question,
                             "chat_history": chat_history})      
    
    return response['answer']

In [7]:
if __name__ == "__main__":
    
    docs = get_docs('https://python.langchain.com/docs/expression_language/')
    vectorStore = create_vector_store(docs)
    chain = create_chain(vectorStore)

    chat_history = [HumanMessage(content='Hello.'),
                    AIMessage(content='Hello. How can I assist you?'),
                    HumanMessage(content='My name is Jim.')]

    while True:
        user_input = input("You: ")
        if user_input.lower() == 'exit':
            break
        response = process_chat(chain, user_input, chat_history)
        chat_history.append(HumanMessage(content=user_input))
        chat_history.append(AIMessage(content=response))
        print("Assistant:", response)

You:  What is LCEL?


Assistant: LCEL stands for LangChain Expression Language. It is designed to provide consistency around behavior and customization over legacy subclasses within the LangChain framework. LCEL allows for the automatic conversion of functions into RunnableLambda objects, enabling the creation of chains that can process inputs in a structured manner. If you have any specific questions about LCEL or its features, feel free to ask!


You:  What is my name?


Assistant: Your name is Jim.


You:  exit
